# NLP 2026
# Lab 3: Classification and BERT

Have you ever read a movie review and wondered:

```“Is this review actually positive or negative?” 🤔```

In this lab, you will build your own sentiment analysis tool using Natural Language Processing (NLP)! Your goal is to automatically classify movie reviews into one of two categories:

✅ Positive

❌ Negative

We will approach this as a binary classification task and you will experiment with increasingly powerful methods — from classic machine learning to modern neural networks based on transformers 🚀

### 🎯 Learning Goals ###

By completing this lab, you should be able to:

- Formulate sentiment analysis as a binary classification problem

- Design and evaluate hand-crafted text features

- Implement a Bag-of-Words representation

- Apply and evaluate Logistic Regression and alternative classifiers

- Understand how BERT tokenization and embeddings work

- Extract sentence representations using: ```CLS``` token, mean token pooling

- Compare classical ML and transformer-based methods

- Critically analyze evaluation metrics beyond accuracy 📊



### Score breakdown

| Exercise            | Points |
|---------------------|--------|
| [Exercise 1](#e1)   | 1      |
| [Exercise 2](#e2)   | 5      |
| [Exercise 3](#e3)   | 5      |
| [Exercise 4](#e4)   | 5      |
| [Exercise 5](#e5)   | 5      |
| [Exercise 6](#e6)   | 5      |
| [Exercise 7](#e7)   | 3      |
| [Exercise 8](#e8)   | 6      |
| [Exercise 9](#e9)   | 5      |
| [Exercise 10](#e10) | 5      |
| [Exercise 12](#e12) | 5      |
| [Exercise 13](#e13) | 10     |
| Total               | 60     |

This score will be scaled down to 0.6 and that will be your final lab score.

### 📌 **Instructions for Delivery** (📅 **Deadline: 23/Feb 18:00**, 🎭 *wildcards possible*)

✅ **Submission Requirements**
+ 📄 You need to submit a **notebook** 📓 with the code, appropriate comments and figures in all questions. Make sure to have a mix of code (some explanations needed if not clear what you implement), figures to support the answers or your claims and proper amount of text to explain your reasoning, answer etc.
+ ⚡ Make sure that **all cells are executed properly** ⚙️ and that **all figures/results/plots** 📊 you include in the report are also visible in your **executed notebook**.
+ You can work on Google Collab (or other environments), but you need to make sure that your delivered notebook is executed properly.

✅ **Collaboration & Integrity**
+ 🗣️ While you may **discuss** the lab with others, you must **write your solutions with your group only**. If you **discuss specific tasks** with others, please **include their names** below.
+ 📜 **Honor Code applies** to this lab. For more details, check **Syllabus §7.2** ⚖️.
+ 📢 **Mandatory Disclosure**:
   - Any **websites** 🌐 (e.g., **Stack Overflow** 💡) or **other resources** used must be **listed and disclosed**.
   - Any **GenAI tools** 🤖 (e.g., **ChatGPT**) used must be **explicitly mentioned**.
   - 🚨 **Failure to disclose these resources is a violation of academic integrity**. See **Syllabus §7.3** for details.

## 0. Setup

We first install the scikit-learn library [Scikit-learn](https://scikit-learn.org/stable/). We will use its classification models.

In [8]:
#pip install -U scikit-learn

We will need [PyTorch](https://pytorch.org/) installed. It is a very popular deep learning library that offers modularized versions of many of the sequence models we discussed in class. It's an important tool that you may want to practice further if you want to dive deeper into NLP, since much of the current academic and industrial research uses it.

Some resources to look further are given below.

* [Documentation](https://pytorch.org/docs/stable/index.html) (We will need this soon)

* [Installation Instructions](https://pytorch.org/get-started/locally/)

* [Quickstart Tutorial](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html)

The cell below should install the library:

In [9]:
 #pip install torch torchvision

The last bit we need is the huggingface transformers library (here is the documentation [https://huggingface.co/docs/transformers/en/index](https://huggingface.co/docs/transformers/en/index)). Transformers are one of the most influential architectures in handling sequences (not only in language). As we discussed in lectures, they excel at taking into account context (which is the salt-and-pepper of NLP) with mechansisms such as self-attetion, which allows them to weigh the importance of different words in a sentence. If you want to know more, revisit the course material (slides and textbook).

We already used huggingface datasets in previous labs and huggingface transformers integrates nicely with that. Apart from the ease of use, huggingface is also providing pre-trained models of different kinds. The list can be found [here](https://huggingface.co/models) ([https://huggingface.co/models](https://huggingface.co/models)). The following line should be enough to install huggingface transformers library:

In [10]:
#pip install transformers

Here, we import the libraries:

In [11]:
import re
from collections import Counter
from string import punctuation

import datasets
import numpy as np
import torch
import tqdm
import transformers
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from torch.utils.data.dataloader import DataLoader
import string
from sklearn.metrics import precision_score, recall_score, f1_score

## 1. Loading the Dataset

We will work with the IMDB dataset [https://huggingface.co/datasets/stanfordnlp/imdb](https://huggingface.co/datasets/stanfordnlp/imdb). It contains the reviews and a label that indicates whether the review is positive or not (the neutral reviews have been filtered out). You can read the paper [here](https://aclanthology.org/P11-1015/).

In [12]:
dataset = datasets.load_dataset('stanfordnlp/imdb', split=['train', 'test'])
print(dataset)

[Dataset({
    features: ['text', 'label'],
    num_rows: 25000
}), Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})]


Notice that the dataset has been loaded as a list of two datasets. They are the `train` and `test` splits that we asked for.
We will use the validation subset to tune the parameters. So, let's split the `train` subset and create a `DatasetDict` object:

In [13]:
train_valid_split = dataset[0].train_test_split(5000)
dataset = datasets.DatasetDict({
    'train': train_valid_split['train'].shuffle(),
    'validation': train_valid_split['test'].shuffle(),
    'test': dataset[1].shuffle(),
})
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 20000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
})


We can print several examples from the `train` dataset:

In [14]:
for i in range(5):
    print('i', i)
    print(dataset['train'][i]['text'])
    print(dataset['train'][i]['label'])
    print()

i 0
When I remember seeing the previews for this movie and not really thinking much about it. It was almost one of those movies that when you see the preview, its stunning, and then when it comes out, you hear nothing and totally miss it, and your memory totally doesn't correct the mistake of missing it. Man On Fire was one of those movies. I was curious on a rental one time, and I decided to take it home with me, my precious Blockbuster rental in my hands. I watched it, and witnessed such a beautiful movie. It is like none other...drama and action combined to create something amazingly spectacular. The cinematography done by Tony Scott is extremely well done and unique, unlike another movie. The subtitles can explain something without even listening to the actual voices, and the music is very intriguing for the setting. I got into this movie, and ended up buying it as soon as I could scurry out of the household and head over to Best Buy. I've watched it several times now. Denzel Washi

Let's extract the labels from the dataset. We will use them to train and evaluate our classifiers.

In [15]:
y_train = dataset['train']['label']
print(y_train)
y_valid = dataset['validation']['label']
print(y_valid)

Column([1, 0, 0, 0, 1, ...])
Column([1, 0, 0, 0, 1, ...])


<a name='e1'></a>
#### Exercise 1: Cleaning the text

(1p) In this exercise you should clean the text in the dataset. This is the same step we saw in the previous labs.

If you think this step is not necessary in this use case, you can skip this step, but make sure to justify your decision.

In [16]:
def clean(text):
    """
    Cleans the text
    Args:
        text: a string that will be cleaned

    Returns: the cleaned text

    """

    # Empty text
    if text == '':
        return text

    ### YOUR CODE HERE
    text = re.sub(r'(?<=\d),(?=\d)', '', text) # remove comma between numbers, i.e. 15,000 -> 15000
    text = re.sub(r'([.,!?;:"])', r' \1 ', text) # space out the punctuation (i.e. "hello, world." -> "hello , world .")

    text = text.lower() # Extra step: making everything lowercase
    text = re.sub(r'http\S+', '', text) # Extra step: removes urls starting with https
    text = re.sub(r'#', '', text) # Extra step: removes hashtags

    # eliminate stop words
    text = re.sub(r'\b(a|an|the|and|or|but)\b', '', text)

    # substitute - with white spaces
    text = re.sub(r'-', ' ', text)

    # remove punctuation
    text = re.sub(r'[^\w\s]', '', text)

    text = re.sub(r'\s+', ' ', text) # remove multiple spaces at the end of the cleaning process, as some of the previous steps can introduce double spacing


    ### YOUR CODE ENDS HERE

    return text


def clean_example(example):
    """
    Applies the clean() function to the example from the Dataset
    Args:
        example: an example from the Dataset

    Returns: update example with cleaned 'text' column

    """
    example['text'] = clean(example['text'])
    return example


dataset = dataset.map(clean_example, desc="Cleaning")
print(dataset)

Cleaning:   0%|          | 0/20000 [00:00<?, ? examples/s]

Cleaning:   0%|          | 0/5000 [00:00<?, ? examples/s]

Cleaning:   0%|          | 0/25000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 20000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
})


## 2. Hand-crafted Features

<a name='e2'></a>
#### Exercise 2: Hand-crafted features

(5p) Write your own hand-crafted feature extraction function. Include at least these types of features:
- length of the text,
- number of different punctuation characters,
- number of positive and negative words.

In [17]:
### YOUR CODE HERE
# you can define the positive and negative words here

positive = ['good', 'excellent', 'amazing', 'groundbreaking', 'awesome', 'captivating', 'mesmerizing', 'thrilling', 'amazed', 'thrilled', 'awe', 'captivated', 'clever', 'critically acclaimed', 'enjoyable', 'enjoyed', 'cheerful', 'underrated', 'interesting', 'interestingly', 'fantastic', 'brilliant', 'masterpiece', 'delightful', 'delight', 'bingable']
negative = ['bad', 'awful', 'disappointing', 'disappointed', 'mediocre', 'uninteresting', 'inexpressive', 'uninterested', 'unpleasant', 'disgusting', 'disgusted', 'awfully', 'poorly', 'catastrophic', 'boring', 'annoying', 'overrated', 'overhyped', 'unwatchable']



def calculate_features(text):
    features = []
    ### YOUR CODE HERE

    # retrieves the length of the text and adds it as a feature
    length = len(text)
    features.append(length)

    # find set of all possible unique punctutation marks
    punct_chars = set(string.punctuation)
    # find all unique punctuation marks are in the text
    punctuation = [char for char in text if char in punct_chars]
    # count the number of different punctuation marks and add it as a feature
    different_punct = len(punctuation)
    features.append(different_punct)

    # find number of positive words in review and add t as a feature
    n_positive = sum([text.count(word) for word in positive])
    features.append(n_positive)

    n_negative = sum([text.count(word) for word in negative])
    features.append(n_negative)

    pos_neg = n_positive - n_negative
    features.append(pos_neg)

    # tokenizr by whitespace
    words = text.split()
    # calculate average word length and add it as a feaure
    words_len = sum(len(word) for word in words)
    avg_word_len = words_len / len(words)
    features.append(avg_word_len)

    # calculate number of capital letters and add it as a feature
    n_cap_words = sum(1 for word in words if word.isupper() and len(word) > 1)
    features.append(n_cap_words)

    ### YOUR CODE ENDS HERE
    return np.array(features, dtype=float)


### YOUR CODE ENDS HERE

The function below will apply your feature extraction implementation to a specified dataset.

In [18]:
def calculate_features_dataset(dataset, features_fn):
    all_features = []
    for e in tqdm.tqdm(dataset, desc='Extracting features'):
        text = e['text']
        features = features_fn(text)
        all_features.append(features)
    all_features = np.array(all_features, dtype=float)
    return all_features

And we can obtain the features for the `train` and `validation` splits. Later you will need to do the same for the `test` subset.

In [19]:
X_train = calculate_features_dataset(dataset['train'], calculate_features)
X_valid = calculate_features_dataset(dataset['validation'], calculate_features)

Extracting features: 100%|██████████| 5000/5000 [00:00<00:00, 16257.47it/s]


### 2.1 Classification

In this section, we will create and train a logistic regression classifier. We will train it on the `train` subset and evaluate on the `validation` split. Later, you will do a final comparison between methods on the `test` subset, but it is important to avoid it when tuning the methods.

In [20]:
classifier = LogisticRegression(solver='lbfgs', max_iter=1000)
classifier.fit(X_train, y_train)

,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",None
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :term:`mul

Let's check the performance on the training data:

In [21]:
print('Features results: train')
pred_train = classifier.predict(X_train)
print(accuracy_score(y_train, pred_train))

Features results: train
0.68975


... and the validation dataset:

In [22]:
print('Features results: validation')
pred_valid = classifier.predict(X_valid)
print(accuracy_score(y_valid, pred_valid))

Features results: validation
0.6916


<a name='e3'></a>
#### Exercise 3: Improving the features

(5p) Iteratively improve your hand-crafted features. Think about what information from the review might be useful for to predict the rating a person gave to the particular movie. You can also look into the expected format (or range) of features for the classifier.

Document the steps you tried (even if unsuccessful) and how they influenced the metrics. Try at least 3 modifications from your original implementation.

In [23]:
### YOUR CODE HERE


### YOUR CODE ENDS HERE

--- YOUR ANSWERS HERE

I first tried adding a feature that calculates the difference between the number of positive and negative words in the review, but this didn't change the accuracy score at all. Probably because that information is already encoded in the number of positive and number of negative words, and adding the difference doesn't add any new information. So the next feature I tried adding was the average word length. This feature increased the accuracy on the validation set. This is likely a useful feature because it can capture the complexity of the language used to describe the movie, which can be an indicator of the review's sentiment. I imagine that people who enjoyed a movie are more likely to put effort into writing a detailed review, however this might not always eb the case since some people are also passionate about writing negative reviews. My next improvement was expanding the positive and negative vocabulary to over double the number of words. I thougt this was most likely the most representative feature in order to determine the sentiment of a review, and I expected it to improve the accuracy score. This change did improve accuracy on both the train set and validation set, but the improvement was not as significant as I expected. Lastly, I extracted the number of words in all caps as a feature, since I thought that this could exprress anger or extreme disappointment. This feature also improved the accuracy score on the train and validation set by a little bit, but not too much. This might be because some people might use all caps to express excitement. The final accuracy on the train set is 0.69115 and on the validation set is 0.6874.

<a name='e4'></a>
#### Exercise 4: Improving the evaluation

(5p) In the previous cells, we only looked at the accuracy of predictions. Investigate which other metrics might be better for our case. You can check the documentation of scikit-learn for evaluation metrics ([https://scikit-learn.org/stable/api/sklearn.metrics.html#classification-metrics](https://scikit-learn.org/stable/api/sklearn.metrics.html#classification-metrics)). Give reasons why the metrics you try can be more informative than raw accuracy score.

Decide which evaluation metric(s) is most suitable for our use-case and give reasons why. Test your features-based classifier and all further classifiers on that metric (apart from the accuracy score).

In [27]:
### YOUR CODE HERE
print('VALIDATION SET')
print('PRECISION')
print(precision_score(y_valid, pred_valid))
print('RECALL')
print(recall_score(y_valid, pred_valid))
print('F1 SCORE')
print(f1_score(y_valid, pred_valid))
print('TRAIN SET')
print('PRECISION')
print(precision_score(y_train, pred_train))
print('RECALL')
print(recall_score(y_train, pred_train))
print('F1 SCORE')
print(f1_score(y_train, pred_train))

### YOUR CODE ENDS HERE

VALIDATION SET
PRECISION
0.6527866752081999
RECALL
0.8165064102564102
F1 SCORE
0.7255250978996084
TRAIN SET
PRECISION
0.6515357000398884
RECALL
0.8163734506197521
F1 SCORE
0.7246994099117086


--- YOUR ANSWERS HERE

we decided to use precision, recall and f1 score as our evaluation metrics because they covers the blindspots of accuracy. Precision measures the proportion of true positives predictions out of all positive predictions, which is useful to avoid false positives. Recall measures the proportion of true positives out of all actual positives, which is useful to avoid false negatives. and the f1 socre is just a combination of precision and recall, which gives us a more balanced view of the model's performance.

## 3. Bag-of-Words Classifier

Similar to the previous lab, we will use the classic bag-of-words representation as one of our embeddings. While it is simple and does not preserve the positions of words, it gives our classifier a lot of useful information.

<a name='e5'></a>
#### Exercise 5: Implementing BOW

(5p) Implement the BOW. In this exercise, we do not give you a rigid structure, so you can conjure your own. The two things your code should produce is the `token_to_id` dictionary, and `bag_of_words()` function that accepts a list of tokens, and the `token_to_id` dictionary while generating the BOW representation as a numpy array.

In [25]:
#### YOUR CODE HERE

MAX_VOCAB_SIZE = 1_000

# The goal is to implement the `bag_of_words(tokens, token_to_id)` function similar to the previous lab.
# You might want to follow the steps:
# - tokenize the `text` column in the dataset,
# - extract the vocabulary from the tokens,
# - limit the vocabulary to `MAX_VOCAB_SIZE`,
# - calculate the `token_to_id` dictionary
# - implement the `bag_of_words(tokens, token_to_id)` function.

token_to_id = {}

def bag_of_words(tokens, token_to_id):
    """
    Creates a bag-of-words representation of the sentence
    Args:
        tokens: a list of tokens
        token_to_id: a dictionary mapping each word to an index in the vocabulary

    Returns:: a numpy array of size vocab_size with the counts of each word in the vocabulary
    """
    pass




#### YOUR CODE ENDS HERE

Here, we will use your implemented function to calculate the bag-of-words for each example in the `train` and `validation` subsets.

In [26]:
train_bows = []
for example in tqdm.tqdm(dataset['train'], desc='Calculating test BOWs'):
    train_bows.append(bag_of_words(example['tokens'], token_to_id))
train_bows = np.array(train_bows, dtype=float)

valid_bows = []
for example in tqdm.tqdm(dataset['validation'], desc='Calculating validation BOWs'):
    valid_bows.append(bag_of_words(example['tokens'], token_to_id))
valid_bows = np.array(valid_bows, dtype=float)

Calculating test BOWs:   0%|          | 0/20000 [00:00<?, ?it/s]


KeyError: 'tokens'

Finally, we can train the classifier on the BOW representations and the labels in the `train` split.

In [ ]:
classifier = LogisticRegression(solver='lbfgs', max_iter=1000)
print('Training classifier...')
classifier.fit(train_bows, y_train)

Let's evaluate the classifier:

In [ ]:
print('BOW results: train')
pred_train = classifier.predict(train_bows)
print(accuracy_score(y_train, pred_train))

print('BOW results: validation')
pred_valid = classifier.predict(valid_bows)
print(accuracy_score(y_valid, pred_valid))

<a name='e6'></a>
#### Exercise 6: Tuning the model

(5p) Try different values for the vocab size. Experiment with adding the hand-crafted features. Test the model on the evaluation metric of your choice (remember to use the validation split).

In [ ]:
#### YOUR CODE HERE


#### YOUR CODE ENDS HERE

## 4. BERT Model

For the first part of this lab, we will be using a pre-trained BERT model from Huggingface, namely the [BERT Cased](https://huggingface.co/google-bert/bert-base-cased). You can read the original paper that introduced this model [here](https://aclanthology.org/N19-1423.pdf). This paper has been one of the most cited papers ever (currently having more than 100,000 citations).

We will specify the model name that can be found on the model's card on huggingface (revisit the first link). Make sure to check what other information Huggingface is offering (e.g. how to use the model, limitations, how to inference, etc.).

In [ ]:
model_name = 'google-bert/bert-base-cased'

### 4.1 Tokenizer

The models on huggingface come with their own tokenizers. They are loaded separately from the models. We can use [AutoTokenizer](https://huggingface.co/docs/transformers/v4.40.2/en/model_doc/auto#transformers.AutoTokenizer)'s `from_pretrained()` method to load it.

Inspect the output: The loaded object is of `BertTokenizer` class. Check the documentation [here](https://huggingface.co/docs/transformers/en/model_doc/bert#transformers.BertTokenizer).

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
print(tokenizer)

Next, let's see how we can use it to tokenize some text.

In [ ]:
print(dataset['test'][0]['text'])
tokenized = tokenizer(dataset['test'][0]['text'], padding=True, return_tensors='pt')
print("---")
print(type(tokenized))
print("---")
print(tokenized)

Examine the outputs: The tokenizer returned three things:
- `input_ids` - this is a PyTorch tensor ([https://pytorch.org/docs/stable/tensors.html](https://pytorch.org/docs/stable/tensors.html)) with the indices of our tokens. PyTorch tensors are similar to numpy arrays. They hold data in a multidimensional array or matrix. The difference is that PyTorch tensors can be placed and modified on the GPU which greatly improves the speed of execution.
- `token_type_ids` - this tensor holds the information about the index of the sentence. This has to do with the classification objective from the original paper, where two sentences were given and the model had to predict if they are connected. Because we only included a single sentence, we have only zeros here. We will not be concerned with it in this lab.
- `attention_mask` - holds the mask that the model will use to determine if the tokens in the `input_ids` are the real tokens or *padding*. Padding is a technique used to ensure that all input sequences have the same length. BERT (like many other NLP models) process data in batches and requires each sequence in a batch to have the same length, so sequences that are shorter than the maximum sequence length in the batch are padded with special tokens. In this case, because we only inputted a single sentence, the mask contains only ones. Later you will see examples where this is not the case.

Let's see how exactly the sentence was tokenized and how we can retrieve the original text. Notice that some words have been split into multiple tokens (remember when we discussed sub-word tokenization in class?). Also pay attention to the added special tokens, namely `CLS` and `SEP`:

The `[CLS]` token is a special classification token added at the beginning of every input sequence. It stands for "classification" (daah!) and is used by BERT to aggregate information from the entire sequence. The final hidden state corresponding to this token (after passing through the transformer layers) is used as the aggregate sequence representation for classification tasks. We will use this later in the lab!

The `[SEP]` token is used to separate different segments or sentences within the input sequence. It stands for "separator" (daaah again!).

In [ ]:
print(tokenized['input_ids'].shape)
print("---")
print(tokenizer.convert_ids_to_tokens(tokenized['input_ids'][0]))
print("---")
print(len(tokenizer.convert_ids_to_tokens(tokenized['input_ids'][0])))
print("---")
print(tokenizer.decode(tokenized['input_ids'][0]))
print("---")
print(tokenizer.decode(tokenized['input_ids'][0], skip_special_tokens=True))

Tokenizer can process a list of sentences. This will create a batched output with tensor's first dimension corresponding to the batch size (the number of sentences we passed to the tokenizer). Examine the following cell and make sure it makes sense to you.

In [ ]:
print(dataset['test'][0:3]['text'])
tokenized = tokenizer(dataset['test'][0:3]['text'], padding=True, return_tensors='pt')
print(tokenized)
print(tokenized['input_ids'].shape)
print(tokenizer.convert_ids_to_tokens(tokenized['input_ids'][0]))
print(len(tokenizer.convert_ids_to_tokens(tokenized['input_ids'][0])))
print(tokenizer.decode(tokenized['input_ids'][0]))
print(tokenizer.decode(tokenized['input_ids'][0], skip_special_tokens=True))

<a name='e7'></a>
#### Exercise 7: Questions about the tokenizer

Answer the following questions:
- (1p) What is the size of the vocabulary?
- (2p) What are the special tokens apart from `[CLS]` and `[SEP]`? What are their functions?

--- YOUR ANSWERS HERE

### 4.2 Loading the Model

In this section, we will load and examine the model. We will start with selecting the device we will place the model on. This will be a GPU (if one is available) or a CPU.

Google Colab offers free access to GPU, provided there is availability (also based on quotas which may vary based on your usage and the overall demand on Colab's resources). If you are working locally, then if you don't have a GPU, CPU will be selected. For the first parts of the assignment running on CPU might be okay but when we have to process the dataset a GPU will be necessary.

The following cell will select the device for us.

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'Device: {device}')

Now, let's load the model from huggingface and move it (slowly because it's heavy due to the large number of parameters) on the device from the previous cell (the methods `to()`).

In [ ]:
model = transformers.AutoModel.from_pretrained(model_name)
print('loaded on device:', model.device)
model.to(device)
print('moved to device', model.device)
print(model)

When loading the model you might have seen the warning about some unexpected weights. This means that the model on huggingface has some additional weights that were downloaded, but our model does not use them. In essence, you can load the same weights (as linked by our `model_name`) to load to different but related models. In our case those would be `BertForMaskedLM` or `BertForNextSentencePrediction` instead of our `BertModel`, which is loaded automatically as the `AutoModel`. Below is a way to load the weights into a different model.

In [ ]:
# transformers.BertForMaskedLM.from_pretrained(model_name)

Next, let's use BERT model for inference. We will tokenize the first sentence of our dataset and pass it to the model. We set `output_hidden_states` to `True` in order to have access to the hidden states of the model. Those represent the latent representations after embedding and transformer layers.

In [ ]:
tokenized = tokenizer(dataset['test'][0]['text'], padding=True, return_tensors='pt').to(device)
print(tokenized)
model_output = model(**tokenized, output_hidden_states=True)

Examine the next cell and make sure everything makes sense to you. Consult the [documentation](https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertModel.forward) in case of doubt.

In [ ]:
print(list(model_output.keys()))
print()
print('pooler_output:')
print(type(model_output['pooler_output']))
print(model_output['pooler_output'].shape)
print()
print('hidden_states:')
print(type(model_output['hidden_states']))
print(len(model_output['hidden_states']))
print(type(model_output['hidden_states'][0]))
print(model_output['hidden_states'][0].shape)
print()
print('last_hidden_state:')
print(type(model_output['last_hidden_state']))
print(model_output['last_hidden_state'].shape)

<a name='e8'></a>
#### Exercise 8: Questions about the Model

Examine the output of the previous cells. Answer the following questions:
- (1p) What is the number of transformer layers in this model?
- (1p) What is the dimension of the embeddings?
- (1p) What is the hidden size of the FFN in the transformer layer?
- (1p) What is the total number of parameters of the model (hint: check the `num_parameters()` method of the model)?
- (1p) How can you find the vocabulary size from the model?
- (1p) What is the length of the `hidden_states` in the output? Why?

--- YOUR ANSWERS HERE

## 5. BERT Sentence Embeddings

Having the model loaded and ready we can work on obtaining the sentence embeddings. During the last lab, you averaged the token embeddings. This time we will start with something else. Remember the CLS token? Its hidden representation is often used for classification as a representation of the whole sentence. We will do exactly that.

But first, we have to tokenize the dataset using BERT tokenizer.

<a name='e9'></a>
#### Exercise 9: BERT tokenizing examples

(5p) Fill in the following function to embed the examples (passed as a parameter) using the tokenizer (also a parameter). The function will tokenize a batch of examples, but the tokenizer can handle that, if you remember from the previous section.

In [ ]:
def tokenize_text_bert(examples, tokenizer):
    """
    Tokenizes the `sentence` column from the batch of examples and returns the whole output of the tokenizer.
    Args:
        examples: a batch of examples
        tokenizer: the BERT tokenizer

    Returns: the tokenized `sentence` column (returns the whole output of the tokenizer)

    """
    ### YOUR CODE HERE
    tokenized_sentence = None



    ### YOUR CODE ENDS HERE
    return tokenized_sentence


In [ ]:
dataset_tokenized_bert = dataset.map(tokenize_text_bert,
                                     fn_kwargs={'tokenizer': tokenizer},
                                     batched=True,
                                     remove_columns=dataset['train'].column_names,)
print(dataset_tokenized_bert)

<a name='e10'></a>
#### Exercise 10: BERT sentence embeddings by the CLS token

(5p) Implement the following function which calculates the sentence embeddings based on the model output (passed to the function as a parameter). It should take the embedding of the CLS token of last layer.

In [ ]:
def calculate_cls_embeddings(input_batch, model_output):
    """
    Calculates the sentence embeddings of a batch of sentences as the last-layer representation of the CLS token.
    Args:
        input_batch: tokenized batch of sentences (as returned by the tokenizer), contains `input_ids`, `token_type_ids`, and `attention_mask` tensors
        model_output: the output of the model given the `input_batch`, contains `last_hidden_state`, `pooler_output`, `hidden_states` tensors

    Returns: tensor of the hidden states of the CLS token (from the last layer) for each example in the batch

    """

    ### YOUR CODE HERE
    sentence_embeddings = None



    ### YOUR CODE ENDS HERE

    return sentence_embeddings

In [ ]:
text = "The weather is nice today."
tokenized = tokenizer(text, padding=True, return_tensors='pt').to(device)
print(tokenized)
model_output = model(**tokenized, output_hidden_states=True)
print(model_output['last_hidden_state'].shape)
sentence_embedding = calculate_cls_embeddings(tokenized, model_output)
print(sentence_embedding.shape)

In [ ]:
def embed_dataset(dataset, model, sentence_embedding_fn, batch_size=8):
    data_collator = transformers.DataCollatorWithPadding(tokenizer)
    data_loader = DataLoader(dataset, batch_size=batch_size, collate_fn=data_collator)
    sentence_embeddings = []
    with torch.no_grad():
        for batch in tqdm.tqdm(data_loader):
            batch.to(device)
            model_output = model(**batch, output_hidden_states=True)
            batch_sentence_embeddings = sentence_embedding_fn(batch, model_output)
            sentence_embeddings.append(batch_sentence_embeddings.detach().cpu())

    sentence_embeddings = torch.concat(sentence_embeddings, dim=0)
    return sentence_embeddings

In [ ]:
bert_cls_train = embed_dataset(dataset_tokenized_bert['train'], model, calculate_cls_embeddings)
print(bert_cls_train.shape)

bert_cls_valid = embed_dataset(dataset_tokenized_bert['validation'], model, calculate_cls_embeddings)
print(bert_cls_valid.shape)

In [ ]:
classifier = LogisticRegression(solver='lbfgs', max_iter=1000)
print('Training classifier...')
classifier.fit(bert_cls_train, y_train)

In [ ]:
print('BERT train')
pred_train = classifier.predict(bert_cls_train)
print(accuracy_score(y_train, pred_train))

print('BERT valid')
pred_valid = classifier.predict(bert_cls_valid)
print(accuracy_score(y_valid, pred_valid))

You can test the model on the evaluation metric of your choice:

In [ ]:
#### YOUR CODE HERE


#### YOUR CODE ENDS HERE

<a name='e11'></a>
#### Exercise 11: BERT Sentence embeddings by averaging tokens

(5p) Implement embedding sentences by averaging the hidden representations of tokens. Make sure to ignore the special and padding tokens. The padding tokens are indicated by the attention mask. You can find the other special tokens using the tokenizer's attributes such as `tokenizer.sep_token_id`. The function accepts the `layer` parameter. Typically, you would use the hidden representations of the last layer, it might be beneficial for some tasks to use previous layers or an averaged representations of multiple layers.

In [ ]:
### YOUR CODE HERE

def calculate_sentence_embeddings(input_batch, model_output, layer=-1):
    """
    Calculates the sentence embeddings of a batch of sentences as a mean of token representations.
    The representations are taken from the layer of the index provided as a `layer` parameter.
    Args:
        input_batch: tokenized batch of sentences (as returned by the tokenizer), contains `input_ids`, `token_type_ids`, and `attention_mask` tensors
        model_output: the output of the model given the `input_batch`, contains `last_hidden_state`, `pooler_output`, `hidden_states` tensors
        layer: specifies the layer of the hidden states that are used to calculate sentence embedding

    Returns: tensor of the averaged hidden states (from the specified layer) for each example in the batch

    """
    attention_mask = input_batch['attention_mask']
    hidden_states = model_output['hidden_states'][layer]

    ### YOUR CODE HERE
    sentence_embeddings = None




    ### YOUR CODE ENDS HERE

    return sentence_embeddings

### YOUR CODE ENDS HERE

We can test it here:


In [ ]:
text = "The weather is nice today."
tokenized = tokenizer(text, padding=True, return_tensors='pt').to(device)
print(tokenized)
model_output = model(**tokenized, output_hidden_states=True)
print(model_output['last_hidden_state'].shape)
sentence_embedding = calculate_sentence_embeddings(tokenized, model_output)
print(sentence_embedding.shape)

We will embed the sentences and evaluate the model on the `validation` subset.


In [ ]:
bert_sentence_train = embed_dataset(dataset_tokenized_bert['train'], model, calculate_sentence_embeddings)
print(bert_cls_train.shape)

bert_sentence_valid = embed_dataset(dataset_tokenized_bert['validation'], model, calculate_sentence_embeddings)
print(bert_cls_valid.shape)

In [ ]:
classifier = LogisticRegression(solver='lbfgs', max_iter=1000)
print('Training classifier...')
classifier.fit(bert_sentence_train, y_train)

In [ ]:
print('BERT train')
pred_train = classifier.predict(bert_sentence_train)
print(accuracy_score(y_train, pred_train))

print('BERT valid')
pred_valid = classifier.predict(bert_sentence_valid)
print(accuracy_score(y_valid, pred_valid))

Test the model on the evaluation metric of your choice:


In [ ]:
#### YOUR CODE HERE


#### YOUR CODE ENDS HERE

## 6. Testing all methods

In this last section, you will bering together all of what you have done so far in this lab. First, you will find the best classifier. Next, you will evaluate all the models you created so far.

<a name='e12'></a>
#### Exercise 12: Find the best classifier for the models

(5p) Basically, do what the title of the exercise says. Evaluate on the `validation` subset. Try at least two other classifiers (apart from the logistic regression). Comment on the results.

In [ ]:
#### YOUR CODE HERE


#### YOUR CODE ENDS HERE

--- YOUR ANSWERS HERE

<a name='e13'></a>
#### Exercise 13: Evaluating methods on the test set

(10p) Test the models you implemented on the test subset:
- Hand-crafted features,
- BOW,
- BERT model based on the CLS token.

You have the models trained already, so only do evaluation.

Evaluate the performance using the metric(s) of your choice. Make sure to discuss the results. Which model performed best? Is this what you expected?

In [ ]:
#### YOUR CODE HERE


### YOUR CODE ENDS HERE

--- YOUR ANSWERS HERE